In [2]:
import pandas as pd
from datetime import datetime

In [4]:
# Loading in the merged datasets (FINISHED PRODUCT AFTER THIS NOTEBOOK)

# Load file with time component
df = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df.csv")

# Load file with time component, categories renamed
df_category = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_category.csv")

# Load file with time component, categories renamed, movie dummy
df_expanded_movies_category = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_expanded_movies_category.csv")

# Load FINAL file with time component, categories renamed, movie dummy, ordered right
df_final = pd.read_csv("/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_final.csv")

# Loading in the data

In [98]:
# Loading seperate datasets

# Define file paths
ratings_file = '/Users/femke/Documents/Uni/DSS/Thesis/Data/ml-1m/ratings.dat'
users_file = '/Users/femke/Documents/Uni/DSS/Thesis/Data/ml-1m/users.dat'
movies_file = '/Users/femke/Documents/Uni/DSS/Thesis/Data/ml-1m/movies.dat'

# Load datasets
ratings = pd.read_csv(ratings_file, sep='::', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')
users = pd.read_csv(users_file, sep='::', header=None, names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], engine='python')
movies = pd.read_csv(movies_file, sep='::', header=None, names=['MovieID', 'Title', 'Genres'], engine='python', encoding='ISO-8859-1')

In [99]:
# Merging the seperate datasets
ratings_plus_users = pd.merge(ratings, users, on='UserID')

# Merge the result with movies dataset
df = pd.merge(ratings_plus_users, movies, on='MovieID')

# First data handling

In [100]:
# Splitting 'Timestamp'
df['Timestamp'] = df['Timestamp'].apply(datetime.fromtimestamp)
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month
df['Day'] = df['Timestamp'].dt.day
df['Hour'] = df['Timestamp'].dt.hour

# Removing the unnessecary features (Zip-code and timestamp)
df = df.drop(columns=["Timestamp", "Zip-code"])

In [120]:
# Creating df_category (Occupation and Age in full for EDA purposes)

# Create a copy of the dataframe
df_category = df.copy()

# Change the names of the categories for the Age feature
age_mapping = {
    1: "Under 18", 18: "18-24", 25: "25-34", 35: "35-44",
    45: "45-49", 50: "50-55", 56: "56+"
}
df_category['Age'] = df_category['Age'].map(age_mapping)

# Change the names of the categories for the Occupation feature
occupation_mapping = {
    0: "Other or not specified", 1: "Academic/educator", 2: "Artist",
    3: "Clerical/admin", 4: "College/grad student", 5: "Customer service",
    6: "Doctor/health care", 7: "Executive/managerial", 8: "Farmer",
    9: "Homemaker", 10: "K-12 student", 11: "Lawyer", 12: "Programmer",
    13: "Retired", 14: "Sales/marketing", 15: "Scientist", 16: "Self-employed",
    17: "Technician/engineer", 18: "Tradesman/craftsman", 19: "Unemployed",
    20: "Writer"
}
df_category['Occupation'] = df_category['Occupation'].map(occupation_mapping)

In [121]:
# Create df_temp (Only occupation in full, preparing for model input)
# Occupation will be a dummy, age will not 

# Create a copy of the dataframe
df_temp = df.copy()

# Change the names of the categories for the Occupation feature
occupation_mapping = {
    0: "Other or not specified", 1: "Academic/educator", 2: "Artist",
    3: "Clerical/admin", 4: "College/grad student", 5: "Customer service",
    6: "Doctor/health care", 7: "Executive/managerial", 8: "Farmer",
    9: "Homemaker", 10: "K-12 student", 11: "Lawyer", 12: "Programmer",
    13: "Retired", 14: "Sales/marketing", 15: "Scientist", 16: "Self-employed",
    17: "Technician/engineer", 18: "Tradesman/craftsman", 19: "Unemployed",
    20: "Writer"
}
df_temp['Occupation'] = df_temp['Occupation'].map(occupation_mapping)

In [122]:
# Splitting the release year from the title and sorting the columns for df_category
df_category['Release_year'] = df_category.Title.str.extract("\((\d{4})\)", expand=True).astype(str)
df_category['Release_year'] = pd.to_datetime(df_category.Release_year, format='%Y')
df_category['Release_year'] = df_category.Release_year.dt.year
df_category['Title'] = df_category.Title.str[:-7]

column_order = ['UserID', 'MovieID', 'Rating', 'Year', 'Month', 'Day', 'Hour','Gender', 'Age', 'Occupation', 
                'Title', 'Release_year', 'Genres']
df_category = df_category[column_order]

In [123]:
# Splitting the release year from the title and sorting the columns for df_temp
df_temp['Release_year'] = df_temp.Title.str.extract("\((\d{4})\)", expand=True).astype(str)
df_temp['Release_year'] = pd.to_datetime(df_temp.Release_year, format='%Y')
df_temp['Release_year'] = df_temp.Release_year.dt.year
df_temp['Title'] = df_temp.Title.str[:-7]

column_order = ['UserID', 'MovieID', 'Rating', 'Year', 'Month', 'Day', 'Hour','Gender', 'Age', 'Occupation', 
                'Title', 'Release_year', 'Genres']
df_temp = df_temp[column_order]

# Creating dummies

In [124]:
# Creating genre dummy for df_category -> df_expanded_movies_category (seperate df so it can be used for EDA purposes)

# Splitting the genres
genres_split = df_category['Genres'].str.get_dummies(sep='|').astype(bool)

# Concatenate the original dataframe with the genres dummy variables
df_expanded_movies_category = pd.concat([df_category.drop(columns=['Genres']), genres_split], axis=1)

In [125]:
# Creating genre dummy for df_temp

# Splitting the genres
genres_split_2 = df_temp['Genres'].str.get_dummies(sep='|').astype(bool)

# Concatenate the original dataframe with the genres dummy variables
df_temp = pd.concat([df_temp.drop(columns=['Genres']), genres_split_2], axis=1)

In [126]:
# One-hot encode Gender
df_temp = pd.get_dummies(df_temp, columns=['Gender'], drop_first=True)
df_temp.rename(columns={'Gender_M': 'Male'}, inplace=True)

In [127]:
# Sorting df_temp
desired_order = ['UserID', 'MovieID', 'Rating', 'Year', 'Month', 'Day', 'Hour', 'Male', 'Age', 'Occupation', 
                   'Title', 'Release_year', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 
                   'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 
                   'Sci-Fi', 'Thriller', 'War', 'Western']

# Reorganize df in temporary dataset
df_temp = df_temp[desired_order]

In [128]:
# One-Hot encode Occupation

# Assuming df_temp is your dataframe
df_temp = pd.get_dummies(df_temp, columns=['Occupation'], drop_first=True)

# Rename the columns to remove the prefix 'Occupation_'
df_temp.columns = [col.replace('Occupation_', '') for col in df_temp.columns]

In [14]:
# Label encode the Age feature
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_temp['Age'] = label_encoder.fit_transform(df_temp['Age'])

In [130]:
# Remove Title column (will be identified by MovieID)
df_temp = df_temp.drop(columns='Title')

In [17]:
# Turn df_temp into df_final
df_final = df_temp.copy()

In [18]:
pd.set_option('display.max_columns', None)
df_final.head()

,UserID,MovieID,Rating,Year,Month,Day,Hour,Male,Age,Release_year,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Artist,Clerical/admin,College/grad student,Customer service,Doctor/health care,Executive/managerial,Farmer,Homemaker,K-12 student,Lawyer,Other or not specified,Programmer,Retired,Sales/marketing,Scientist,Self-employed,Technician/engineer,Tradesman/craftsman,Unemployed,Writer
0,1,1193,5,2000,12,31,23,False,0,1975,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,2,1193,5,2000,12,31,22,True,6,1975,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
2,12,1193,4,2000,12,31,0,True,2,1975,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,15,1193,4,2000,12,30,19,True,2,1975,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,17,1193,5,2000,12,30,7,True,5,1975,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [27]:
df_final.head()

,UserID,MovieID,Rating,Year,Month,Day,Hour,Male,Age,Release_year,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Artist,Clerical/admin,College/grad student,Customer service,Doctor/health care,Executive/managerial,Farmer,Homemaker,K-12 student,Lawyer,Other or not specified,Programmer,Retired,Sales/marketing,Scientist,Self-employed,Technician/engineer,Tradesman/craftsman,Unemployed,Writer
0,1,1193,5,2000,12,31,23,False,0,1975,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,2,1193,5,2000,12,31,22,True,6,1975,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
2,12,1193,4,2000,12,31,0,True,2,1975,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,15,1193,4,2000,12,30,19,True,2,1975,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,17,1193,5,2000,12,30,7,True,5,1975,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


# Scaling the numerical features (ONLY RUN THIS AFTER FEATURE ENGINEERING)

In [21]:
df_temp = df_final.copy()

UserID                    int64
MovieID                   int64
Rating                    int64
Year                      int64
Month                     int64
Day                       int64
Hour                      int64
Male                       bool
Age                       int64
Release_year              int64
Action                     bool
Adventure                  bool
Animation                  bool
Children's                 bool
Comedy                     bool
Crime                      bool
Documentary                bool
Drama                      bool
Fantasy                    bool
Film-Noir                  bool
Horror                     bool
Musical                    bool
Mystery                    bool
Romance                    bool
Sci-Fi                     bool
Thriller                   bool
War                        bool
Western                    bool
Artist                     bool
Clerical/admin             bool
College/grad student       bool
Customer

In [22]:
from sklearn.preprocessing import StandardScaler

# List of features to scale
features_to_scale = ['Rating', 'Year', 'Month', 'Day', 'Hour', 'Release_year', 'Age']

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the features
df_temp[features_to_scale] = scaler.fit_transform(df_temp[features_to_scale])


In [23]:
df_temp.head()

,UserID,MovieID,Rating,Year,Month,Day,Hour,Male,Age,Release_year,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Artist,Clerical/admin,College/grad student,Customer service,Doctor/health care,Executive/managerial,Farmer,Homemaker,K-12 student,Lawyer,Other or not specified,Programmer,Retired,Sales/marketing,Scientist,Self-employed,Technician/engineer,Tradesman/craftsman,Unemployed,Writer
0,1,1193,1.269747,-0.298761,1.2101,1.753776,1.434634,False,-1.839831,-0.815259,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,2,1193,1.269747,-0.298761,1.2101,1.753776,1.309130,True,2.574357,-0.815259,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
2,12,1193,0.374573,-0.298761,1.2101,1.753776,-1.451967,True,-0.368435,-0.815259,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,15,1193,0.374573,-0.298761,1.2101,1.641197,0.932617,True,-0.368435,-0.815259,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,17,1193,1.269747,-0.298761,1.2101,1.641197,-0.573436,True,1.838659,-0.815259,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


# Check for missing values

In [145]:
missing_values_null = df_final.isnull().sum()
print(missing_values_null)

UserID                    0
MovieID                   0
Rating                    0
Year                      0
Month                     0
Day                       0
Hour                      0
Male                      0
Age                       0
Release_year              0
Action                    0
Adventure                 0
Animation                 0
Children's                0
Comedy                    0
Crime                     0
Documentary               0
Drama                     0
Fantasy                   0
Film-Noir                 0
Horror                    0
Musical                   0
Mystery                   0
Romance                   0
Sci-Fi                    0
Thriller                  0
War                       0
Western                   0
Artist                    0
Clerical/admin            0
College/grad student      0
Customer service          0
Doctor/health care        0
Executive/managerial      0
Farmer                    0
Homemaker           

In [146]:
# Check for missing values (NaN or None)
missing_values_NaN_None = df_final.isna().sum()
print(missing_values_NaN_None)

UserID                    0
MovieID                   0
Rating                    0
Year                      0
Month                     0
Day                       0
Hour                      0
Male                      0
Age                       0
Release_year              0
Action                    0
Adventure                 0
Animation                 0
Children's                0
Comedy                    0
Crime                     0
Documentary               0
Drama                     0
Fantasy                   0
Film-Noir                 0
Horror                    0
Musical                   0
Mystery                   0
Romance                   0
Sci-Fi                    0
Thriller                  0
War                       0
Western                   0
Artist                    0
Clerical/admin            0
College/grad student      0
Customer service          0
Doctor/health care        0
Executive/managerial      0
Farmer                    0
Homemaker           

No missing values -> however, not every user rated every movie, could be considered missing values

# Saving the files

In [143]:
# Saving all dataframes
# Save file with time component
df.to_csv('/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df.csv', index=False)

# Save file with time component, categories renamed
df_category.to_csv('/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_category.csv', index=False)

# Save file with time component, categories renamed, movie dummy
df_expanded_movies_category.to_csv('/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_expanded_movies_category.csv', index=False)

# Save file with time component, categories renamed, movie dummy, ordered right
df_final.to_csv('/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_final.csv', index=False)


In [28]:
df_final.to_csv('/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_final.csv', index=False)


# OLD CODE 


In [ ]:
# Save file with time component, movie dummy
df_expanded_movies.to_csv('/Users/femke/Documents/Uni/DSS/Thesis/Data/ml1m_df_expanded_movies.csv', index=False)

In [ ]:
# #OLD WAY TO GET TO df_expanded_movies_category and explore the movies dataset

# # Splitting the release year from the title for movies dataset
# movies['Release_year'] = movies.Title.str.extract("\((\d{4})\)", expand=True).astype(str)
# movies['Release_year'] = pd.to_datetime(movies.Release_year, format='%Y')
# movies['Release_year'] = movies.Release_year.dt.year
# movies['Title'] = movies.Title.str[:-7]

# # Separating the genres and creating the dummies
# # Creating all unique genres
# genres_unique = pd.DataFrame(movies.Genres.str.split('|').tolist()).stack().unique()
# genres_unique = pd.DataFrame(genres_unique, columns=['Genre'])

# # Remove old Genres column and get dummies
# movies = movies.join(movies.Genres.str.get_dummies().astype(bool))
# movies.drop('Genres', inplace=True, axis=1)

# # Merge expanded movie dataset (with dummies) with ratings and user dataset
# df_expanded_movies = pd.merge(ratings_plus_users, movies, on='MovieID')

# # Creating time components in df_expanded_movies
# df_expanded_movies['Timestamp'] = df_expanded_movies['Timestamp'].apply(datetime.fromtimestamp)
# df_expanded_movies['Year'] = df_expanded_movies['Timestamp'].dt.year
# df_expanded_movies['Month'] = df_expanded_movies['Timestamp'].dt.month
# df_expanded_movies['Day'] = df_expanded_movies['Timestamp'].dt.day
# df_expanded_movies['Hour'] = df_expanded_movies['Timestamp'].dt.hour

# # Creating df_expanded_movies_category (Occupation and Age in full + movie dummy)

# # Create a copy of the dataframe
# df_expanded_movies_category = df_expanded_movies.copy()

# # Change the names of the categories for the Age feature
# age_mapping = {
#     1: "Under 18", 18: "18-24", 25: "25-34", 35: "35-44",
#     45: "45-49", 50: "50-55", 56: "56+"
# }
# df_expanded_movies_category['Age'] = df_expanded_movies_category['Age'].map(age_mapping)

# # Change the names of the categories for the Occupation feature
# occupation_mapping = {
#     0: "other or not specified", 1: "academic/educator", 2: "artist",
#     3: "clerical/admin", 4: "college/grad student", 5: "customer service",
#     6: "doctor/health care", 7: "executive/managerial", 8: "farmer",
#     9: "homemaker", 10: "K-12 student", 11: "lawyer", 12: "programmer",
#     13: "retired", 14: "sales/marketing", 15: "scientist", 16: "self-employed",
#     17: "technician/engineer", 18: "tradesman/craftsman", 19: "unemployed",
#     20: "writer"
# }
# df_expanded_movies_category['Occupation'] = df_expanded_movies_category['Occupation'].map(occupation_mapping)

# # Sorting df_expanded_movies_category
# desired_columns = ['UserID', 'MovieID', 'Rating', 'Timestamp', 'Year', 'Month', 'Day', 'Hour','Gender', 'Age', 'Occupation', 'Zip-code', 
#                    'Title', 'Release_year', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 
#                    'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 
#                    'Sci-Fi', 'Thriller', 'War', 'Western']

# # Now, we can reorder the dataframe and drop the 'Timestamp' column
# df_expanded_movies_category = df_expanded_movies_category[desired_columns]